In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
import numpy as np
import re

In [2]:
from google.colab import drive
drive.mount("/content/drive")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
dossier='/content/drive/MyDrive/'
df=pd.read_csv(dossier+'text_preprocessed_boubacar.csv')
df=df.astype(str).sample(df.shape[0]//100)
df_questions=df["Content"]
df_answers=df["Answers"]

In [ ]:
df

,Unnamed: 0,Title,Content,Answers
228797,228797,pygame.sprite.group.add error,ok peice code lives_group = pygame.sprite.grou...,do not forget to call parent s constructor cla...
374319,374319,django static files working heroku,django application working well local server ....,this is my setting for static files to deploy ...
92393,92393,possible variants zip python,"example , code looks like : = [ 1 , 2 ] b = [ ...",if you are interested only in the result then ...
199476,199476,following loop work ?,"def escape_html ( ) : ( , ) ( ( `` & '' , '' &...",for i o in amp gt lt quot i and o are your loo...
155204,155204,include .pyd module files py2exe compilation,trying compile python script . executing exe g...,py exe let us you specify additional python mo...
...,...,...,...,...
168305,168305,minimax explanation `` dummies '',quite new algorithms trying understand minimax...,the idea of minimax is that there in a two pla...
159426,159426,"given list tuples , return new list first valu...",list tuples want new list consisting first val...,how about map lambda e e e for e in a
271249,271249,customize styles django clearablefileinput widget,use imagefield django model image upload facil...,just make your own input class and alter the r...
308610,308610,python : selenium phantomjs empty page source,trouble selenium phantomjs windows7 want get s...,by default phantomjs use sslv but many sites a...


In [ ]:
df.head()


,Unnamed: 0,Title,Content,Answers
228797,228797,pygame.sprite.group.add error,ok peice code lives_group = pygame.sprite.grou...,do not forget to call parent s constructor cla...
374319,374319,django static files working heroku,django application working well local server ....,this is my setting for static files to deploy ...
92393,92393,possible variants zip python,"example , code looks like : = [ 1 , 2 ] b = [ ...",if you are interested only in the result then ...
199476,199476,following loop work ?,"def escape_html ( ) : ( , ) ( ( `` & '' , '' &...",for i o in amp gt lt quot i and o are your loo...
155204,155204,include .pyd module files py2exe compilation,trying compile python script . executing exe g...,py exe let us you specify additional python mo...


In [ ]:
questions_train, questions_test, answers_train, answers_test = train_test_split(df_questions,df_answers, test_size=0.2, random_state=42)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

# Créer un tokenizer pour convertir les phrases en séquences de mots
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions_train)

# Convertir les phrases en séquences de mots
questions_train = tokenizer.texts_to_sequences(questions_train)
questions_test = tokenizer.texts_to_sequences(questions_test)


# Créer un tokenizer pour convertir les phrases en séquences de mots
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(answers_train)

# Convertir les phrases en séquences de mots
answers_train = tokenizer.texts_to_sequences(answers_train)
answers_test = tokenizer.texts_to_sequences(answers_test)

max_length_questions = max([len(s) for s in questions_train])
max_length_answers = max([len(s) for s in answers_train])
max_length=max([max_length_questions, max_length_answers])
# Pad les séquences pour qu'elles aient toutes la même longueur

questions_train = tf.keras.preprocessing.sequence.pad_sequences(questions_train, maxlen=max_length)
questions_test = tf.keras.preprocessing.sequence.pad_sequences(questions_test, maxlen=max_length)


# Pad les séquences pour qu'elles aient toutes la même longueur

answers_train = tf.keras.preprocessing.sequence.pad_sequences(answers_train, maxlen=max_length)
answers_test = tf.keras.preprocessing.sequence.pad_sequences(answers_test, maxlen=max_length)

# Créer un encoder-decoder avec un LSTM
encoder_inputs = Input(shape=(max_length,1))
encoder = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length_questions)(encoder_inputs)
encoder = LSTM(64, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_length,))
decoder_embedding = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(tokenizer.word_index)+1, activation='softmax')
outputs = decoder_dense(decoder_outputs)


In [ ]:

# Compiler le modèle
model = Model([encoder_inputs, decoder_inputs], [outputs])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Entraîner le modèle
model.fit([questions_train, questions_train], answers_train, epochs=10, batch_size=8, validation_split=0.2, steps_per_epoch=2, validation_steps=2)


Epoch 1/10
2/2 [==============================] - 10s 2s/step - loss: 9.4781 - accuracy: 2.1310e-04 - val_loss: 9.4673 - val_accuracy: 0.9752
Epoch 2/10
2/2 [==============================] - 1s 996ms/step - loss: 9.4631 - accuracy: 0.9745 - val_loss: 9.4494 - val_accuracy: 0.9761
Epoch 3/10
2/2 [==============================] - 1s 998ms/step - loss: 9.4439 - accuracy: 0.9710 - val_loss: 9.4236 - val_accuracy: 0.9765
Epoch 4/10
2/2 [==============================] - 1s 1s/step - loss: 9.4147 - accuracy: 0.9713 - val_loss: 9.3804 - val_accuracy: 0.9766
Epoch 5/10
2/2 [==============================] - 1s 1s/step - loss: 9.3629 - accuracy: 0.9805 - val_loss: 9.2983 - val_accuracy: 0.9769
Epoch 6/10
2/2 [==============================] - 1s 985ms/step - loss: 9.2646 - accuracy: 0.9768 - val_loss: 9.1420 - val_accuracy: 0.9770
Epoch 7/10
2/2 [==============================] - 1s 986ms/step - loss: 9.0912 - accuracy: 0.9656 - val_loss: 8.9109 - val_accuracy: 0.9771
Epoch 8/10
2/2 [========

In [ ]:
# Évaluer le modèle sur les données de test
loss, acc = model.evaluate([questions_test, questions_test], answers_test, verbose=0, batch_size=8)
print("Test accuracy:", acc)

Test accuracy: 0.9756694436073303


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
from nltk.stem import *
def stem_traincorpus(data):
    stemmer = PorterStemmer()
    out_data=""
    for words in data:
        out_data+= stemmer.stem(words)
    return out_data
def remove_stopwords(data):
  text=data.lower()
  text_tokens = word_tokenize(text)
  tokens_without_sw = [word for word in text_tokens if not word in stop_words]
  return (" ").join(tokens_without_sw)

In [ ]:
import keras
#def prediction
def predict_text(#define input text and model
                 input_text, tokenizer, model, 
                 #define tokenizer maximum length of sequence
                 maxlen_seq, padding = 'post', truncating = 'post'
                 ):
    
    #prediction
    text = str(input_text)
    sequence = tokenizer.texts_to_sequences([text])
    sequence = pad_sequences(sequence, maxlen = maxlen_seq)
    predict = model.predict([sequence,sequence])
    
    return predict
tokenizer = tf.keras.preprocessing.text.Tokenizer()
new_text=predict_text("print message screen",tokenizer,model,max_length)
print(new_text)
a=tokenizer.sequences_to_texts([new_text])
#tableau=np.array([questions_train, answers_train])
#tableau.reshape(1,tableau.shape[1],tableau.shape[2])

1/1 [==============================] - 1s 853ms/step
[[[8.6511209e-05 8.4118052e-05 8.4208732e-05 ... 7.6546989e-05
   7.6107572e-05 7.5759548e-05]
  [9.1995309e-05 8.7892586e-05 8.7928871e-05 ... 7.6746102e-05
   7.6006596e-05 7.5522621e-05]
  [9.8424272e-05 9.2195070e-05 9.2196002e-05 ... 7.6941666e-05
   7.5871612e-05 7.5288757e-05]
  ...
  [3.1213902e-04 2.0819058e-04 2.1008721e-04 ... 8.0171114e-05
   7.3313880e-05 7.0628084e-05]
  [3.1213902e-04 2.0819058e-04 2.1008721e-04 ... 8.0171114e-05
   7.3313880e-05 7.0628084e-05]
  [3.1213902e-04 2.0819058e-04 2.1008721e-04 ... 8.0171114e-05
   7.3313880e-05 7.0628084e-05]]]


TypeError: ignored